# 01 — Data Simulation: Building a PlayStation Player Ecosystem

## Why Simulated Data?

This project models **Customer Lifetime Value** and **Next Best Action** for PlayStation's player base.
Since real PlayStation telemetry isn't publicly available, we build a synthetic dataset designed to mirror
the data structures and behavioural patterns that PlayStation's DSAE team would work with in production.

This isn't random data. Every table, column, and distribution is designed around a core question:
**what observable signals predict whether a player will churn, upgrade, or buy?**

The simulation embeds realistic behavioural patterns — declining engagement before churn, free DLC
engagement predicting paid DLC conversion, trophy depth as a retention signal — so that models trained
on this data face the same analytical challenges as models trained on real production data.

### Data Architecture

We simulate 7 interconnected tables that mirror a typical gaming data warehouse:

| Table | Description | Key Fields |
|-------|-------------|------------|
| `players` | 5,000 player profiles | signup date, country, age, platform |
| `subscriptions` | PS Plus history | tier (Free/Essential/Extra/Premium), auto-renew, billing cycle |
| `sessions` | Game play logs | game played, duration, date |
| `purchases` | Store transactions | full games, paid DLC, free DLC claims, microtransactions |
| `trophies` | Achievement unlocks | game, trophy type (bronze/silver/gold/platinum) |
| `games` | 30-title catalog | genre, price, first-party flag, PS Plus availability |
| `dlc_catalog` | 13 DLC items | parent game, free vs paid, price |

### Embedded Behavioural Signals

The following patterns are baked into the simulation. Our models in later notebooks must
discover these from observable data alone:

| Signal | Business Relevance | How It's Simulated |
|--------|-------------------|-------------------|
| **Churn decay** | Players don't cancel suddenly — engagement drops first | Session frequency and duration decline in the 1-6 months before cancellation |
| **Auto-renew as intent** | Turning off auto-renew is the strongest churn predictor | 85% of churners have auto-renew off vs 20% of retained players |
| **Free DLC &rarr; Paid DLC pipeline** | Free content creates purchase intent | Players who claim Valhalla (free DLC) get a 50% boost to paid DLC purchase probability |
| **Trophy depth = stickiness** | Completionists are less likely to churn | Trophy earn rate correlates with session volume; both inversely correlate with churn |
| **Tier commitment** | Annual subscribers are more loyal | Annual billing reduces churn probability by 50% vs monthly |
| **Seasonal patterns** | Gaming peaks around holidays | December sessions are 30% above baseline; summer is 15% below |

## Setup

Only standard libraries needed — `pandas` and `numpy`.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

N_PLAYERS = 5_000
OBSERVATION_START = pd.Timestamp("2024-01-01")
OBSERVATION_END = pd.Timestamp("2025-12-31")
SEED = 42

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

print(f"Simulating {N_PLAYERS:,} players over {OBSERVATION_START.date()} to {OBSERVATION_END.date()}")

Simulating 5,000 players over 2024-01-01 to 2025-12-31


## Game & DLC Catalog

A mix of 30 titles spanning Sony first-party flagships (God of War, Spider-Man, Horizon),
major third-party releases (Elden Ring, Baldur's Gate 3), free-to-play staples (Fortnite, GTA Online),
and PS Plus catalog titles. The DLC catalog includes 13 items — critically, God of War: Ragnarok's
**Valhalla DLC is free**, which creates our key analytical signal: does engaging with free DLC
predict willingness to pay for future DLC?

In [2]:
GAMES = [
    # First-party flagships
    {"game_id": "G001", "game_name": "God of War: Ragnarok",         "genre": "Action-RPG",   "release_date": "2022-11-09", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G002", "game_name": "Spider-Man 2",                 "genre": "Action",       "release_date": "2023-10-20", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G003", "game_name": "Horizon Forbidden West",       "genre": "Action-RPG",   "release_date": "2022-02-18", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": True},
    {"game_id": "G004", "game_name": "The Last of Us Part I",        "genre": "Action",       "release_date": "2022-09-02", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": True},
    {"game_id": "G005", "game_name": "Gran Turismo 7",               "genre": "Racing",       "release_date": "2022-03-04", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": True},
    {"game_id": "G006", "game_name": "Returnal",                     "genre": "Roguelike",    "release_date": "2021-04-30", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": True},
    {"game_id": "G007", "game_name": "Ratchet & Clank: Rift Apart",  "genre": "Platformer",   "release_date": "2021-06-11", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": True},
    {"game_id": "G008", "game_name": "Astro Bot",                    "genre": "Platformer",   "release_date": "2024-09-06", "is_first_party": True,  "base_price": 59.99, "in_ps_plus_extra": False},
    {"game_id": "G009", "game_name": "Helldivers 2",                 "genre": "Shooter",      "release_date": "2024-02-08", "is_first_party": True,  "base_price": 39.99, "in_ps_plus_extra": False},
    {"game_id": "G010", "game_name": "Stellar Blade",                "genre": "Action",       "release_date": "2024-04-26", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": False},
    # Major third-party
    {"game_id": "G011", "game_name": "FIFA 24",                      "genre": "Sports",       "release_date": "2023-09-29", "is_first_party": False, "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G012", "game_name": "Call of Duty: MW III",         "genre": "FPS",          "release_date": "2023-11-10", "is_first_party": False, "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G013", "game_name": "Elden Ring",                   "genre": "Action-RPG",   "release_date": "2022-02-25", "is_first_party": False, "base_price": 59.99, "in_ps_plus_extra": False},
    {"game_id": "G014", "game_name": "Baldur's Gate 3",              "genre": "RPG",          "release_date": "2023-09-06", "is_first_party": False, "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G015", "game_name": "Hogwarts Legacy",              "genre": "Action-RPG",   "release_date": "2023-02-10", "is_first_party": False, "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G016", "game_name": "Diablo IV",                    "genre": "Action-RPG",   "release_date": "2023-06-06", "is_first_party": False, "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G017", "game_name": "Final Fantasy XVI",            "genre": "Action-RPG",   "release_date": "2023-06-22", "is_first_party": False, "base_price": 69.99, "in_ps_plus_extra": False},
    {"game_id": "G018", "game_name": "Resident Evil 4 Remake",       "genre": "Horror",       "release_date": "2023-03-24", "is_first_party": False, "base_price": 59.99, "in_ps_plus_extra": False},
    {"game_id": "G019", "game_name": "GTA Online",                   "genre": "Action",       "release_date": "2014-11-18", "is_first_party": False, "base_price": 0.00,  "in_ps_plus_extra": False},
    {"game_id": "G020", "game_name": "Fortnite",                     "genre": "Battle Royale", "release_date": "2017-07-21", "is_first_party": False, "base_price": 0.00,  "in_ps_plus_extra": False},
    # Catalog / PS Plus filler
    {"game_id": "G021", "game_name": "Ghost of Tsushima",            "genre": "Action",       "release_date": "2020-07-17", "is_first_party": True,  "base_price": 59.99, "in_ps_plus_extra": True},
    {"game_id": "G022", "game_name": "Demon's Souls",                "genre": "Action-RPG",   "release_date": "2020-11-12", "is_first_party": True,  "base_price": 69.99, "in_ps_plus_extra": True},
    {"game_id": "G023", "game_name": "Uncharted: Legacy of Thieves", "genre": "Action",       "release_date": "2022-01-28", "is_first_party": True,  "base_price": 49.99, "in_ps_plus_extra": True},
    {"game_id": "G024", "game_name": "Death Stranding DC",           "genre": "Action",       "release_date": "2022-03-28", "is_first_party": True,  "base_price": 49.99, "in_ps_plus_extra": True},
    {"game_id": "G025", "game_name": "Stardew Valley",               "genre": "Simulation",   "release_date": "2016-02-26", "is_first_party": False, "base_price": 14.99, "in_ps_plus_extra": True},
    {"game_id": "G026", "game_name": "Hades",                        "genre": "Roguelike",    "release_date": "2021-08-13", "is_first_party": False, "base_price": 24.99, "in_ps_plus_extra": True},
    {"game_id": "G027", "game_name": "Cyberpunk 2077",               "genre": "RPG",          "release_date": "2020-12-10", "is_first_party": False, "base_price": 59.99, "in_ps_plus_extra": False},
    {"game_id": "G028", "game_name": "It Takes Two",                 "genre": "Co-op",        "release_date": "2021-03-26", "is_first_party": False, "base_price": 39.99, "in_ps_plus_extra": True},
    {"game_id": "G029", "game_name": "Dead Cells",                   "genre": "Roguelike",    "release_date": "2018-08-07", "is_first_party": False, "base_price": 24.99, "in_ps_plus_extra": True},
    {"game_id": "G030", "game_name": "Bloodborne",                   "genre": "Action-RPG",   "release_date": "2015-03-24", "is_first_party": True,  "base_price": 19.99, "in_ps_plus_extra": True},
]

DLC_CATALOG = [
    {"dlc_id": "D001", "game_id": "G001", "dlc_name": "Valhalla (Free DLC)",           "is_free": True,  "price": 0.00,  "release_date": "2023-12-12"},
    {"dlc_id": "D002", "game_id": "G003", "dlc_name": "Burning Shores",                "is_free": False, "price": 19.99, "release_date": "2023-04-19"},
    {"dlc_id": "D003", "game_id": "G002", "dlc_name": "New Game+ Update",              "is_free": True,  "price": 0.00,  "release_date": "2024-03-07"},
    {"dlc_id": "D004", "game_id": "G013", "dlc_name": "Shadow of the Erdtree",         "is_free": False, "price": 39.99, "release_date": "2024-06-21"},
    {"dlc_id": "D005", "game_id": "G017", "dlc_name": "Echoes of the Fallen",          "is_free": False, "price": 9.99,  "release_date": "2023-12-07"},
    {"dlc_id": "D006", "game_id": "G017", "dlc_name": "The Rising Tide",               "is_free": False, "price": 19.99, "release_date": "2024-04-18"},
    {"dlc_id": "D007", "game_id": "G016", "dlc_name": "Vessel of Hatred",              "is_free": False, "price": 39.99, "release_date": "2024-10-08"},
    {"dlc_id": "D008", "game_id": "G009", "dlc_name": "Steeled Veterans Warbond",      "is_free": False, "price": 9.99,  "release_date": "2024-03-14"},
    {"dlc_id": "D009", "game_id": "G009", "dlc_name": "Democratic Detonation Warbond", "is_free": False, "price": 9.99,  "release_date": "2024-05-09"},
    {"dlc_id": "D010", "game_id": "G021", "dlc_name": "Legends Mode (Free Update)",    "is_free": True,  "price": 0.00,  "release_date": "2020-10-16"},
    {"dlc_id": "D011", "game_id": "G027", "dlc_name": "Phantom Liberty",               "is_free": False, "price": 29.99, "release_date": "2023-09-26"},
    {"dlc_id": "D012", "game_id": "G005", "dlc_name": "Free Content Update Pack 1",    "is_free": True,  "price": 0.00,  "release_date": "2023-05-25"},
    {"dlc_id": "D013", "game_id": "G018", "dlc_name": "Separate Ways",                 "is_free": False, "price": 9.99,  "release_date": "2023-09-21"},
]

games_df = pd.DataFrame(GAMES)
games_df["release_date"] = pd.to_datetime(games_df["release_date"])

dlc_df = pd.DataFrame(DLC_CATALOG)
dlc_df["release_date"] = pd.to_datetime(dlc_df["release_date"])

print(f"Game catalog: {len(games_df)} titles ({games_df['is_first_party'].sum()} first-party)")
print(f"DLC catalog:  {len(dlc_df)} items ({dlc_df['is_free'].sum()} free, {(~dlc_df['is_free']).sum()} paid)")
games_df[["game_id", "game_name", "genre", "is_first_party", "base_price"]].head(10)

Game catalog: 30 titles (15 first-party)
DLC catalog:  13 items (4 free, 9 paid)


,game_id,game_name,genre,is_first_party,base_price
0,G001,God of War: Ragnarok,Action-RPG,True,69.99
1,G002,Spider-Man 2,Action,True,69.99
2,G003,Horizon Forbidden West,Action-RPG,True,69.99
3,G004,The Last of Us Part I,Action,True,69.99
4,G005,Gran Turismo 7,Racing,True,69.99
5,G006,Returnal,Roguelike,True,69.99
6,G007,Ratchet & Clank: Rift Apart,Platformer,True,69.99
7,G008,Astro Bot,Platformer,True,59.99
8,G009,Helldivers 2,Shooter,True,39.99
9,G010,Stellar Blade,Action,True,69.99


## Player Archetypes — The Hidden Structure

In any subscription/engagement business, players aren't homogeneous. PlayStation's player base
naturally segments into behavioural clusters with very different value and risk profiles.

We define 6 latent archetypes that drive the simulation. These are **invisible to our models** —
they won't appear in the exported data. The whole point of the modelling work in later notebooks
is to rediscover these segments from observable behaviour.

| Archetype | Share | Churn Risk | Play Pattern | Revenue | PS Plus Tier |
|-----------|-------|------------|-------------|---------|-------------|
| **Whale** | 8% | Very low | Daily, long sessions | Heavy — buys most DLC, frequent MTX | Premium |
| **Core Gamer** | 25% | Low-medium | Regular, 3-4x/week | Moderate — selective purchases | Extra |
| **Casual** | 35% | Medium | Weekend warrior, short sessions | Light — occasional buys | Essential |
| **Lapsed** | 15% | High | Was active, now declining | Minimal — stopped spending | Essential (at risk) |
| **Dormant** | 10% | Very high | Nearly inactive | Zero | Essential/Free |
| **New Player** | 7% | Uncertain | Initial enthusiasm burst | Variable — still forming habits | Mixed |

This distribution reflects a common pattern in gaming: a small percentage of high-value players
(whales + core) generate the majority of revenue, while a long tail of casual and at-risk players
represent either growth opportunity or retention risk.

In [3]:
ARCHETYPES = {
    "whale": {
        "weight": 0.08, "churn_base_prob": 0.03,
        "sessions_per_month_mean": 22, "session_duration_mean": 90,
        "monthly_spend_mean": 45.0, "upgrade_propensity": 0.85,
        "dlc_purchase_prob": 0.70, "free_dlc_claim_prob": 0.90,
        "trophy_rate": 0.35, "game_diversity": 12,
        "first_party_affinity": 0.60, "platform_ps5_prob": 0.80,
    },
    "core_gamer": {
        "weight": 0.25, "churn_base_prob": 0.08,
        "sessions_per_month_mean": 14, "session_duration_mean": 60,
        "monthly_spend_mean": 20.0, "upgrade_propensity": 0.55,
        "dlc_purchase_prob": 0.40, "free_dlc_claim_prob": 0.70,
        "trophy_rate": 0.20, "game_diversity": 8,
        "first_party_affinity": 0.50, "platform_ps5_prob": 0.60,
    },
    "casual": {
        "weight": 0.35, "churn_base_prob": 0.15,
        "sessions_per_month_mean": 6, "session_duration_mean": 35,
        "monthly_spend_mean": 8.0, "upgrade_propensity": 0.20,
        "dlc_purchase_prob": 0.15, "free_dlc_claim_prob": 0.40,
        "trophy_rate": 0.08, "game_diversity": 4,
        "first_party_affinity": 0.35, "platform_ps5_prob": 0.40,
    },
    "lapsed": {
        "weight": 0.15, "churn_base_prob": 0.45,
        "sessions_per_month_mean": 3, "session_duration_mean": 25,
        "monthly_spend_mean": 3.0, "upgrade_propensity": 0.10,
        "dlc_purchase_prob": 0.08, "free_dlc_claim_prob": 0.20,
        "trophy_rate": 0.03, "game_diversity": 2,
        "first_party_affinity": 0.30, "platform_ps5_prob": 0.35,
    },
    "dormant": {
        "weight": 0.10, "churn_base_prob": 0.65,
        "sessions_per_month_mean": 1, "session_duration_mean": 15,
        "monthly_spend_mean": 0.5, "upgrade_propensity": 0.05,
        "dlc_purchase_prob": 0.03, "free_dlc_claim_prob": 0.10,
        "trophy_rate": 0.01, "game_diversity": 1,
        "first_party_affinity": 0.25, "platform_ps5_prob": 0.25,
    },
    "new_player": {
        "weight": 0.07, "churn_base_prob": 0.20,
        "sessions_per_month_mean": 10, "session_duration_mean": 50,
        "monthly_spend_mean": 15.0, "upgrade_propensity": 0.30,
        "dlc_purchase_prob": 0.20, "free_dlc_claim_prob": 0.50,
        "trophy_rate": 0.12, "game_diversity": 3,
        "first_party_affinity": 0.45, "platform_ps5_prob": 0.70,
    },
}

COUNTRIES = {
    "US": 0.30, "UK": 0.12, "JP": 0.15, "DE": 0.08, "FR": 0.07,
    "BR": 0.06, "CA": 0.05, "AU": 0.04, "ES": 0.03, "IT": 0.03,
    "MX": 0.02, "KR": 0.02, "NL": 0.01, "SE": 0.01, "PL": 0.01,
}

AGE_GROUPS = {"13-17": 0.08, "18-24": 0.28, "25-34": 0.35, "35-44": 0.20, "45+": 0.09}

print("Archetypes defined. Distribution:")
for name, params in ARCHETYPES.items():
    print(f"  {name:<12s}: {params['weight']*100:.0f}%  |  churn={params['churn_base_prob']:.0%}  sessions/mo={params['sessions_per_month_mean']}  spend=${params['monthly_spend_mean']}")

Archetypes defined. Distribution:
  whale       : 8%  |  churn=3%  sessions/mo=22  spend=$45.0
  core_gamer  : 25%  |  churn=8%  sessions/mo=14  spend=$20.0
  casual      : 35%  |  churn=15%  sessions/mo=6  spend=$8.0
  lapsed      : 15%  |  churn=45%  sessions/mo=3  spend=$3.0
  dormant     : 10%  |  churn=65%  sessions/mo=1  spend=$0.5
  new_player  : 7%  |  churn=20%  sessions/mo=10  spend=$15.0


## Generate Player Profiles

Each player gets an archetype assignment, then individual-level parameters are sampled by adding
~20-30% noise around the archetype baseline. This means two 'whales' won't behave identically —
just like real players within a segment show natural variation.

Demographics (country, age, platform) roughly mirror PlayStation's global market distribution.

In [4]:
rng = np.random.default_rng(SEED)

arch_names = list(ARCHETYPES.keys())
arch_weights = [ARCHETYPES[a]["weight"] for a in arch_names]
archetypes = rng.choice(arch_names, size=N_PLAYERS, p=arch_weights)

countries = rng.choice(list(COUNTRIES.keys()), size=N_PLAYERS, p=list(COUNTRIES.values()))
age_groups = rng.choice(list(AGE_GROUPS.keys()), size=N_PLAYERS, p=list(AGE_GROUPS.values()))
platforms = [
    "PS5" if rng.random() < ARCHETYPES[arch]["platform_ps5_prob"] else "PS4"
    for arch in archetypes
]

signup_dates = []
for arch in archetypes:
    if arch == "new_player":
        offset = rng.integers(0, 300)
        signup_dates.append(pd.Timestamp("2025-01-01") + pd.Timedelta(days=int(offset)))
    else:
        offset = rng.integers(0, 2190)
        signup_dates.append(pd.Timestamp("2019-01-01") + pd.Timedelta(days=int(offset)))

player_ids = [f"PSN-{i:05d}" for i in range(N_PLAYERS)]

player_params = {}
for i, pid in enumerate(player_ids):
    base = ARCHETYPES[archetypes[i]]
    player_params[pid] = {
        "archetype": archetypes[i],
        "churn_base_prob":     np.clip(base["churn_base_prob"] * rng.normal(1.0, 0.25), 0.01, 0.90),
        "sessions_per_month":  max(0, int(base["sessions_per_month_mean"] * rng.normal(1.0, 0.3))),
        "session_duration_mean": max(10, base["session_duration_mean"] * rng.normal(1.0, 0.2)),
        "monthly_spend_mean":  max(0, base["monthly_spend_mean"] * rng.normal(1.0, 0.3)),
        "upgrade_propensity":  np.clip(base["upgrade_propensity"] * rng.normal(1.0, 0.2), 0, 1),
        "dlc_purchase_prob":   np.clip(base["dlc_purchase_prob"] * rng.normal(1.0, 0.25), 0, 1),
        "free_dlc_claim_prob": np.clip(base["free_dlc_claim_prob"] * rng.normal(1.0, 0.2), 0, 1),
        "trophy_rate":         np.clip(base["trophy_rate"] * rng.normal(1.0, 0.3), 0, 0.8),
        "game_diversity":      max(1, int(base["game_diversity"] * rng.normal(1.0, 0.3))),
        "first_party_affinity": np.clip(base["first_party_affinity"] * rng.normal(1.0, 0.2), 0, 1),
    }

players_df = pd.DataFrame({
    "player_id": player_ids, "signup_date": signup_dates,
    "country": countries, "age_group": age_groups,
    "platform": platforms, "archetype": archetypes,
})

print(f"Players generated: {len(players_df):,}")
print(f"\nArchetype distribution:\n{players_df['archetype'].value_counts().to_string()}")
print(f"\nPlatform: {players_df['platform'].value_counts().to_dict()}")
players_df.head()

Players generated: 5,000

Archetype distribution:
archetype
casual        1728
core_gamer    1319
lapsed         733
dormant        494
whale          386
new_player     340

Platform: {'PS4': 2618, 'PS5': 2382}


,player_id,signup_date,country,age_group,platform,archetype
0,PSN-00000,2024-05-25,US,35-44,PS5,lapsed
1,PSN-00001,2023-06-11,FR,35-44,PS5,casual
2,PSN-00002,2020-03-29,BR,18-24,PS4,dormant
3,PSN-00003,2024-12-20,US,13-17,PS4,lapsed
4,PSN-00004,2024-12-13,US,18-24,PS5,core_gamer


## Generate PS Plus Subscriptions

PS Plus tier assignment is driven by each player's `upgrade_propensity` — whales gravitate toward
Premium, casuals toward Essential or Free. Three key signals are embedded here:

1. **auto_renew** — the single strongest churn indicator. In our simulation, 85% of churners
   had auto-renew turned off before cancelling, vs only 20% of retained subscribers.
2. **Billing cycle** — annual subscribers churn at half the rate of monthly subscribers.
   Committed payment = committed player.
3. **Tenure protection** — players who subscribed very recently (<3 months) are less likely
   to have churned yet, reflecting the 'honeymoon period' in real subscription data.

In [5]:
rng_sub = np.random.default_rng(SEED + 1)
sub_records = []

for _, row in players_df.iterrows():
    pid = row["player_id"]
    params = player_params[pid]
    signup = row["signup_date"]

    up = params["upgrade_propensity"]
    roll = rng_sub.random()
    if roll < (1 - up) * 0.4:
        tier = "Free"
    elif roll < (1 - up * 0.5):
        tier = "Essential"
    elif roll < (1 - up * 0.15):
        tier = "Extra"
    else:
        tier = "Premium"

    if tier == "Free":
        sub_records.append({
            "player_id": pid, "tier": "Free", "start_date": signup,
            "end_date": None, "is_active": True,
            "auto_renew": False, "billing_cycle": "none",
        })
        continue

    max_offset = max(30, min(365, (OBSERVATION_END - signup).days // 2))
    sub_start = signup + pd.Timedelta(days=int(rng_sub.integers(0, max_offset)))
    if sub_start > OBSERVATION_END:
        sub_start = signup

    if params["churn_base_prob"] < 0.10:
        cycle_p = [0.15, 0.20, 0.65]
    elif params["churn_base_prob"] < 0.30:
        cycle_p = [0.40, 0.30, 0.30]
    else:
        cycle_p = [0.60, 0.25, 0.15]
    billing_cycle = rng_sub.choice(["monthly", "quarterly", "annual"], p=cycle_p)

    churn_prob = params["churn_base_prob"]
    if billing_cycle == "annual":
        churn_prob *= 0.5
    months_active = (OBSERVATION_END - sub_start).days / 30
    if months_active < 3:
        churn_prob *= 0.3

    churned = rng_sub.random() < churn_prob
    auto_renew = rng_sub.random() < (0.15 if churned else 0.80)

    end_date = None
    if churned:
        max_churn = max(31, (OBSERVATION_END - sub_start).days)
        end_date = sub_start + pd.Timedelta(days=int(rng_sub.integers(30, max_churn)))
        if end_date > OBSERVATION_END:
            end_date = OBSERVATION_END - pd.Timedelta(days=int(rng_sub.integers(1, 60)))

    sub_records.append({
        "player_id": pid, "tier": tier, "start_date": sub_start,
        "end_date": end_date, "is_active": not churned,
        "auto_renew": auto_renew, "billing_cycle": billing_cycle,
    })

subs_df = pd.DataFrame(sub_records)

print(f"Subscription records: {len(subs_df):,}")
print(f"\nTier distribution:\n{subs_df['tier'].value_counts().to_string()}")
print(f"\nChurn rate by tier:")
for tier in ["Essential", "Extra", "Premium"]:
    t = subs_df[subs_df["tier"] == tier]
    print(f"  {tier}: {(~t['is_active']).mean():.1%}")
print(f"\nAuto-renew (active): {subs_df[subs_df['is_active'] & (subs_df['tier'] != 'Free')]['auto_renew'].mean():.1%}")
print(f"Auto-renew (churned): {subs_df[~subs_df['is_active']]['auto_renew'].mean():.1%}")

Subscription records: 5,000

Tier distribution:
tier
Essential    2845
Free         1357
Extra         565
Premium       233

Churn rate by tier:
  Essential: 18.5%
  Extra: 9.6%
  Premium: 14.6%

Auto-renew (active): 79.9%
Auto-renew (churned): 15.1%


## Generate Game Sessions

Session data is the behavioural backbone of this dataset. Each player's monthly session
count and duration are driven by their archetype, with two important modulations:

**Seasonal patterns** — gaming activity peaks in December (+30% above baseline) and dips
in summer (-15%). This mirrors real industry patterns where holiday releases and school
breaks drive engagement spikes.

**Churn decay** — this is the most important signal for our churn model. Players who
eventually cancel don't just drop off a cliff. Their engagement erodes gradually:
activity is 70% of normal 6 months before churn, drops to 50% at 3 months, 30% at 2 months,
and just 15% in the final month. This realistic decay pattern means our model can potentially
detect churn risk *before* the player cancels — which is exactly when intervention matters.

In [6]:
rng_sess = np.random.default_rng(SEED + 10)
MONTHS = pd.date_range(OBSERVATION_START, OBSERVATION_END, freq="MS")

churn_dates = {}
for _, sub in subs_df.iterrows():
    if not sub["is_active"] and sub["end_date"] is not None:
        churn_dates[sub["player_id"]] = pd.Timestamp(sub["end_date"])

SEASONAL = {
    1: 1.15, 2: 1.05, 3: 1.00, 4: 0.95, 5: 0.90, 6: 0.85,
    7: 0.85, 8: 0.90, 9: 1.00, 10: 1.05, 11: 1.15, 12: 1.30,
}

def select_player_games(params, games_df, rng):
    """Pick games this player engages with based on diversity & first-party affinity."""
    n = min(params["game_diversity"], len(games_df))
    weights = np.ones(len(games_df))
    fp = games_df["is_first_party"].values
    weights[fp] *= (1 + params["first_party_affinity"])
    weights[~fp] *= (1 + (1 - params["first_party_affinity"]))
    weights[games_df["base_price"].values == 0] *= 1.5
    weights /= weights.sum()
    idx = rng.choice(len(games_df), size=n, replace=False, p=weights)
    return games_df.iloc[idx]["game_id"].tolist()

session_records = []
print("Generating sessions...")
for idx, (_, player) in enumerate(players_df.iterrows()):
    if idx % 1000 == 0:
        print(f"  Player {idx:,} / {N_PLAYERS:,}")

    pid = player["player_id"]
    params = player_params[pid]
    signup = player["signup_date"]
    player_games = select_player_games(params, games_df, rng_sess)
    churn_date = churn_dates.get(pid)

    for month in MONTHS:
        if month < signup.replace(day=1):
            continue
        if churn_date and month > churn_date:
            continue

        mult = SEASONAL[month.month]
        if churn_date:
            months_until = (churn_date - month).days / 30
            if months_until <= 1:     mult *= 0.15
            elif months_until <= 2:   mult *= 0.30
            elif months_until <= 3:   mult *= 0.50
            elif months_until <= 6:   mult *= 0.70
        mult *= rng_sess.normal(1.0, 0.1)
        mult = max(0.05, mult)

        n_sessions = max(0, int(params["sessions_per_month"] * mult))
        days_in_month = (month + pd.offsets.MonthEnd(0)).day

        for _ in range(n_sessions):
            day = int(rng_sess.integers(1, days_in_month + 1))
            duration = max(5, int(rng_sess.normal(
                params["session_duration_mean"] * mult,
                params["session_duration_mean"] * 0.3
            )))
            session_records.append({
                "player_id": pid, "game_id": rng_sess.choice(player_games),
                "session_date": month.replace(day=day),
                "duration_minutes": duration,
            })

sessions_df = pd.DataFrame(session_records)
print(f"\nTotal sessions: {len(sessions_df):,}")
print(f"Avg duration: {sessions_df['duration_minutes'].mean():.0f} min")
print(f"Sessions per player (mean): {len(sessions_df) / N_PLAYERS:.0f}")

Generating sessions...
  Player 0 / 5,000
  Player 1,000 / 5,000
  Player 2,000 / 5,000
  Player 3,000 / 5,000
  Player 4,000 / 5,000

Total sessions: 809,870
Avg duration: 61 min
Sessions per player (mean): 162


## Generate Purchases & DLC

Three revenue streams, each with distinct analytical value:

**Full game purchases** — frequency and recency of game buying is a direct input to CLV.
Older games are discounted (50% off after 12 months) to reflect real pricing behaviour.

**DLC (free & paid)** — this is where the **Valhalla signal** lives. God of War: Ragnarok's
Valhalla DLC was released for free in December 2023. In our simulation, players who claim
this free DLC receive a **50% boost** to their paid DLC purchase probability. This models
a real-world dynamic: free content creates engagement and purchase intent. Detecting this
pattern is exactly the kind of insight PlayStation's personalisation team would act on.

**Microtransactions** — for F2P titles (GTA Online, Fortnite). High MTX spend often
indicates a player who values the platform but may not be on a paid PS Plus tier —
a potential upgrade target.

In [7]:
rng_purch = np.random.default_rng(SEED + 20)
purchase_records = []

print("Generating purchases...")
for idx, (_, player) in enumerate(players_df.iterrows()):
    if idx % 1000 == 0:
        print(f"  Player {idx:,} / {N_PLAYERS:,}")

    pid = player["player_id"]
    params = player_params[pid]
    signup = player["signup_date"]
    churn_date = churn_dates.get(pid)
    player_games = select_player_games(params, games_df, rng_purch)

    # ---- Full game purchases ----
    n_buys = max(0, int(rng_purch.poisson(params["monthly_spend_mean"] / 15)))
    paid_games = games_df[games_df["base_price"] > 0]["game_id"].tolist()
    bought_games = set()

    for gid in rng_purch.choice(paid_games, size=min(n_buys, len(paid_games)), replace=False):
        game = games_df[games_df["game_id"] == gid].iloc[0]
        earliest = max(signup, game["release_date"], OBSERVATION_START)
        latest = churn_date if churn_date else OBSERVATION_END
        if earliest >= latest:
            continue
        offset = int(rng_purch.integers(0, max(1, (latest - earliest).days)))
        pdate = earliest + pd.Timedelta(days=offset)
        months_old = (pdate - game["release_date"]).days / 30
        discount = 1.0 if months_old < 6 else (0.7 if months_old < 12 else 0.5)
        purchase_records.append({
            "player_id": pid, "item_id": gid, "item_type": "full_game",
            "purchase_date": pdate, "amount": round(game["base_price"] * discount, 2),
        })
        bought_games.add(gid)

    # ---- DLC (free claims + paid purchases) ----
    this_player_valhalla = False

    for _, dlc in dlc_df.iterrows():
        if dlc["game_id"] not in player_games and dlc["game_id"] not in bought_games:
            continue
        earliest = max(signup, dlc["release_date"], OBSERVATION_START)
        latest = churn_date if churn_date else OBSERVATION_END
        if earliest >= latest:
            continue

        if dlc["is_free"]:
            if rng_purch.random() < params["free_dlc_claim_prob"]:
                offset = int(rng_purch.integers(0, max(1, (latest - earliest).days)))
                purchase_records.append({
                    "player_id": pid, "item_id": dlc["dlc_id"], "item_type": "dlc_free",
                    "purchase_date": earliest + pd.Timedelta(days=offset), "amount": 0.00,
                })
                if dlc["dlc_id"] == "D001":
                    this_player_valhalla = True
        else:
            buy_prob = params["dlc_purchase_prob"]
            if this_player_valhalla:
                buy_prob = min(buy_prob * 1.5, 0.85)  # Valhalla signal!
            if rng_purch.random() < buy_prob:
                offset = int(rng_purch.integers(0, max(1, min(90, (latest - earliest).days))))
                purchase_records.append({
                    "player_id": pid, "item_id": dlc["dlc_id"], "item_type": "dlc_paid",
                    "purchase_date": earliest + pd.Timedelta(days=offset), "amount": dlc["price"],
                })

    # ---- Microtransactions (F2P games) ----
    for gid in ["G019", "G020"]:
        if gid in player_games:
            n_mtx = rng_purch.poisson(params["monthly_spend_mean"] / 10)
            for _ in range(n_mtx):
                earliest = max(signup, OBSERVATION_START)
                latest = churn_date if churn_date else OBSERVATION_END
                if earliest >= latest:
                    continue
                offset = int(rng_purch.integers(0, max(1, (latest - earliest).days)))
                purchase_records.append({
                    "player_id": pid, "item_id": gid, "item_type": "mtx",
                    "purchase_date": earliest + pd.Timedelta(days=offset),
                    "amount": float(rng_purch.choice([4.99, 9.99, 19.99, 24.99], p=[.4, .3, .2, .1])),
                })

purchases_df = pd.DataFrame(purchase_records)

print(f"\nTotal transactions: {len(purchases_df):,}")
print(f"By type:\n{purchases_df['item_type'].value_counts().to_string()}")
print(f"\nTotal revenue: ${purchases_df['amount'].sum():,.2f}")

# ---- Valhalla signal check ----
val_claimers = set(purchases_df[purchases_df["item_id"] == "D001"]["player_id"])
paid_dlc_buyers = set(purchases_df[purchases_df["item_type"] == "dlc_paid"]["player_id"])
v_to_p = len(val_claimers & paid_dlc_buyers)
non_val = set(purchases_df["player_id"]) - val_claimers
nv_to_p = len(non_val & paid_dlc_buyers)
print(f"\n--- Valhalla Signal Check ---")
print(f"Valhalla claimers: {len(val_claimers):,}")
print(f"  -> bought paid DLC: {v_to_p:,} ({v_to_p/max(1,len(val_claimers))*100:.1f}%)")
print(f"Non-claimers: {len(non_val):,}")
print(f"  -> bought paid DLC: {nv_to_p:,} ({nv_to_p/max(1,len(non_val))*100:.1f}%)")

Generating purchases...
  Player 0 / 5,000
  Player 1,000 / 5,000
  Player 2,000 / 5,000
  Player 3,000 / 5,000
  Player 4,000 / 5,000

Total transactions: 13,430
By type:
item_type
full_game    4257
mtx          4160
dlc_paid     2991
dlc_free     2022

Total revenue: $242,775.57

--- Valhalla Signal Check ---
Valhalla claimers: 496
  -> bought paid DLC: 335 (67.5%)
Non-claimers: 2,731
  -> bought paid DLC: 1,223 (44.8%)


## Generate Trophies

Trophy data serves as a measure of **engagement depth**. A player can have many sessions but
earn few trophies (casual play), or fewer sessions but earn many trophies (focused, completionist
play). This distinction matters for both churn prediction and propensity modelling:

- **Churn signal**: Trophy earn rate dropping over time indicates disengagement even if the player
  is still technically logging in.
- **Propensity signal**: Trophy hunters (especially platinum earners) are completionists who are
  more likely to buy DLC to extend a game they love.

In [8]:
rng_troph = np.random.default_rng(SEED + 30)
trophy_records = []

sess_counts = sessions_df.groupby(["player_id", "game_id"]).size().reset_index(name="n_sessions")

# Pre-build lookup for performance
session_dates_lookup = sessions_df.groupby(["player_id", "game_id"])["session_date"].apply(list).to_dict()

print("Generating trophies...")
for _, row in sess_counts.iterrows():
    pid = row["player_id"]
    params = player_params[pid]
    n_sess = row["n_sessions"]

    expected = n_sess * params["trophy_rate"]
    n_trophies = min(rng_troph.poisson(max(0.1, expected)), 50)
    if n_trophies == 0:
        continue

    dates = session_dates_lookup.get((pid, row["game_id"]), [])
    if not dates:
        continue

    for _ in range(n_trophies):
        trophy_records.append({
            "player_id": pid, "game_id": row["game_id"],
            "trophy_date": rng_troph.choice(dates),
            "trophy_type": rng_troph.choice(
                ["bronze", "silver", "gold", "platinum"], p=[0.65, 0.20, 0.10, 0.05]),
        })

trophies_df = pd.DataFrame(trophy_records)
print(f"\nTotal trophies: {len(trophies_df):,}")
print(f"By type:\n{trophies_df['trophy_type'].value_counts().to_string()}")

Generating trophies...

Total trophies: 158,707
By type:
trophy_type
bronze      103286
silver       31512
gold         15944
platinum      7965


## Save to CSV

We export 7 tables plus a hidden archetype key (for model validation only).
The `archetype` column is dropped from the players table — our models must
infer player segments from behaviour alone.

In [9]:
players_export = players_df.drop(columns=["archetype"])
archetype_map = players_df[["player_id", "archetype"]].copy()

tables = {
    "players":        players_export,
    "subscriptions":  subs_df,
    "games":          games_df,
    "dlc_catalog":    dlc_df,
    "sessions":       sessions_df,
    "purchases":      purchases_df,
    "trophies":       trophies_df,
    "_archetype_key": archetype_map,
}

print("Saving to CSV...\n")
for name, df in tables.items():
    path = DATA_DIR / f"{name}.csv"
    df.to_csv(path, index=False)
    size_kb = path.stat().st_size / 1024
    print(f"  {name + '.csv':<25s}  {len(df):>8,} rows  |  {size_kb:>8.1f} KB")

print(f"\nDone! All files saved to {DATA_DIR.resolve()}")

Saving to CSV...

  players.csv                   5,000 rows  |     165.2 KB
  subscriptions.csv             5,000 rows  |     240.3 KB
  games.csv                        30 rows  |       1.7 KB
  dlc_catalog.csv                  13 rows  |       0.7 KB
  sessions.csv                809,870 rows  |   23028.6 KB
  purchases.csv                13,430 rows  |     511.6 KB
  trophies.csv                158,707 rows  |    5099.0 KB
  _archetype_key.csv            5,000 rows  |      89.6 KB

Done! All files saved to /Users/amunif/Downloads/PS_Churn_Propensity_Project/data


## Sanity Checks

Before moving to feature engineering, we verify the data behaves as expected.

In [10]:
print("=" * 60)
print("SANITY CHECKS")
print("=" * 60)

assert set(players_df["player_id"]) == set(subs_df["player_id"])
print("OK: Every player has a subscription record")

for pid, cdate in churn_dates.items():
    player_sess = sessions_df[sessions_df["player_id"] == pid]
    if len(player_sess) > 0:
        max_sess = player_sess["session_date"].max()
        assert max_sess <= cdate + pd.Timedelta(days=31)
print("OK: No sessions significantly after churn dates")

paid_subs = subs_df[subs_df["tier"] != "Free"]
overall_churn = (~paid_subs["is_active"]).mean()
print(f"OK: Overall paid churn rate = {overall_churn:.1%}")

whale_ids = set(archetype_map[archetype_map["archetype"] == "whale"]["player_id"])
casual_ids = set(archetype_map[archetype_map["archetype"] == "casual"]["player_id"])
whale_spend = purchases_df[purchases_df["player_id"].isin(whale_ids)].groupby("player_id")["amount"].sum().mean()
casual_spend = purchases_df[purchases_df["player_id"].isin(casual_ids)].groupby("player_id")["amount"].sum().mean()
print(f"OK: Whale avg spend ${whale_spend:.2f} vs Casual ${casual_spend:.2f}")

active_ar = subs_df[subs_df["is_active"] & (subs_df["tier"] != "Free")]["auto_renew"].mean()
churned_ar = subs_df[~subs_df["is_active"]]["auto_renew"].mean()
print(f"OK: Auto-renew active={active_ar:.0%} vs churned={churned_ar:.0%}")

print("\nAll checks passed! Ready for 02_player_360_features.")

SANITY CHECKS
OK: Every player has a subscription record
OK: No sessions significantly after churn dates
OK: Overall paid churn rate = 16.9%
OK: Whale avg spend $206.44 vs Casual $36.83
OK: Auto-renew active=80% vs churned=15%

All checks passed! Ready for 02_player_360_features.
